Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [90]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [91]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [92]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  #print(dataset[0])
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [93]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [94]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases  = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print(hidden.get_shape().as_list()) # [16, 14, 14, 16]

    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print(hidden.get_shape().as_list()) # [16, 7, 7, 16]

    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape.get_shape().as_list()) # [16, 784]

    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print(hidden.get_shape().as_list())

    return tf.matmul(hidden, layer4_weights) + layer4_biases


  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 14, 14, 16]
[16, 7, 7, 16]
[16, 784]
[16, 64]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]
[10000, 14, 14, 16]
[10000, 7, 7, 16]
[10000, 784]
[10000, 64]


In [95]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')

  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.156358
Minibatch accuracy: 12.5%
Validation accuracy: 9.7%
Minibatch loss at step 500: 1.006369
Minibatch accuracy: 56.2%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.460345
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 1.236296
Minibatch accuracy: 81.2%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.883243
Minibatch accuracy: 68.8%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 0.528218
Minibatch accuracy: 81.2%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.634424
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Test accuracy: 92.3%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [96]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases  = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases  = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 7 * image_size // 7 * depth, num_hidden], stddev=0.1))
  layer3_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    pool = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    print(pool.get_shape().as_list()) # [16, 14, 14, 16]

    conv = tf.nn.conv2d(pool, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print(hidden.get_shape().as_list()) # [16, 7, 7, 16]

    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print(hidden.get_shape().as_list()) # [16, 4, 4, 16]

    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape.get_shape().as_list()) # [16, 256]

    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print(hidden.get_shape().as_list())

    return tf.matmul(hidden, layer4_weights) + layer4_biases


  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 14, 14, 1]
[16, 7, 7, 16]
[16, 4, 4, 16]
[16, 256]
[16, 64]
[10000, 14, 14, 1]
[10000, 7, 7, 16]
[10000, 4, 4, 16]
[10000, 256]
[10000, 64]
[10000, 14, 14, 1]
[10000, 7, 7, 16]
[10000, 4, 4, 16]
[10000, 256]
[10000, 64]


In [97]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')

  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.432380
Minibatch accuracy: 25.0%
Validation accuracy: 9.9%
Minibatch loss at step 500: 1.003518
Minibatch accuracy: 56.2%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 0.632133
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 1.308965
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.865564
Minibatch accuracy: 68.8%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 0.655418
Minibatch accuracy: 68.8%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.555392
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Minibatch loss at step 3500: 0.451696
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 0.268521
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Test accuracy: 92.2%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.
http://www.cs.cmu.edu/~aarti/Class/10701_Spring14/slides/DeepLearning.pdf

---

In [146]:
batch_size = 128
patch_size = 5
depth1 = 6
depth3 = 16
depth5 = 120
num_hidden = 84 # number of hidden nodes in layer 6 

# l2 regularization
regularization_beta = 1e-3
# dropout in full connection layer
keep_prob = 0.8
# when used dropout or batchnorm, we can tune the learning rate lagger to detec a better result
learning_rate = 0.17

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)

  #keep_prob = tf.placeholder(tf.float32)

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases  = tf.Variable(tf.zeros([depth1]))

  layer3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth3], stddev=0.1))
  layer3_biases  = tf.Variable(tf.constant(1.0, shape=[depth3]))

  layer5_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth3, depth5], stddev=0.1))
  layer5_biases  = tf.Variable(tf.constant(1.0, shape=[depth5]))

  layer6_weights = tf.Variable(tf.truncated_normal([120, num_hidden], stddev=0.1))
  layer6_biases  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  output_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  output_biases  = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') # 4 dim strides
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    print(hidden1.get_shape().as_list()) # [batch_size, 28, 28, 6] # batch, in_height, in_width, in_channels

    pool2 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID') # value, ksize, strides, padding...
    print(pool2.get_shape().as_list()) # [batch_size, 14, 14, 6]

    conv3 = tf.nn.conv2d(pool2, layer3_weights, [1, 1, 1, 1], padding='VALID')
    hidden3 = tf.nn.relu(conv3 + layer3_biases)
    print(hidden3.get_shape().as_list()) # [batch_size, 10, 10, 10]

    pool4 = tf.nn.max_pool(hidden3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID') # value, ksize, strides, padding...
    print(pool4.get_shape().as_list()) # [batch_size, 5, 5, 16]

    conv5 = tf.nn.conv2d(pool4, layer5_weights, [1, 1, 1, 1], padding='VALID')
    hidden5 = tf.nn.relu(conv5 + layer5_biases)
    print(hidden5.get_shape().as_list()) # [batch_size, 1, 1, 120]

    shape = hidden5.get_shape().as_list()
    reshape = tf.reshape(hidden5, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape.get_shape().as_list()) # [batch_size, 120]

    hidden6 = tf.nn.relu(tf.matmul(reshape, layer6_weights) + layer6_biases)
    hidden6 = tf.nn.dropout(hidden6, keep_prob)
    print(hidden6.get_shape().as_list()) # [batch_size, 84]

    return (tf.matmul(hidden6, output_weights) + output_biases, layer1_weights, layer3_weights, layer5_weights, layer6_weights)

  # Training computation.
  logits, layer1_weights, layer3_weights, layer5_weights, layer6_weights = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        + regularization_beta * (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer3_weights)
                                               + tf.nn.l2_loss(layer5_weights) + tf.nn.l2_loss(layer6_weights)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset)[0])
  test_prediction = tf.nn.softmax(model(tf_test_dataset)[0])

[128, 28, 28, 6]
[128, 14, 14, 6]
[128, 10, 10, 16]
[128, 5, 5, 16]
[128, 1, 1, 120]
[128, 120]
[128, 84]
[10000, 28, 28, 6]
[10000, 14, 14, 6]
[10000, 10, 10, 16]
[10000, 5, 5, 16]
[10000, 1, 1, 120]
[10000, 120]
[10000, 84]
[10000, 28, 28, 6]
[10000, 14, 14, 6]
[10000, 10, 10, 16]
[10000, 5, 5, 16]
[10000, 1, 1, 120]
[10000, 120]
[10000, 84]


In [149]:
num_steps = 10001

import time

with tf.Session(graph=graph) as session:
  start = time.clock()
  tf.initialize_all_variables().run()
  print('Initialized')

  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 4.634295
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.896213
Minibatch accuracy: 81.2%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 0.475018
Minibatch accuracy: 90.6%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 0.536859
Minibatch accuracy: 86.7%
Validation accuracy: 88.1%
Minibatch loss at step 4000: 0.496260
Minibatch accuracy: 89.1%
Validation accuracy: 88.5%
Minibatch loss at step 5000: 0.548573
Minibatch accuracy: 83.6%
Validation accuracy: 89.0%
Minibatch loss at step 6000: 0.247704
Minibatch accuracy: 95.3%
Validation accuracy: 89.0%
Minibatch loss at step 7000: 0.427846
Minibatch accuracy: 88.3%
Validation accuracy: 89.1%
Minibatch loss at step 8000: 0.283377
Minibatch accuracy: 93.0%
Validation accuracy: 89.5%
Minibatch loss at step 9000: 0.279347
Minibatch accuracy: 94.5%
Validation accuracy: 89.4%
Minibatch loss at step 10000: 0.364292
Minibatch accuracy: 90.6%
Validation accura